In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os

import arviz as az

/opt/miniconda3/envs/env4eeg/lib/python3.13/site-packages/arviz/__init__.py:39: FutureWarning: 
ArviZ is undergoing a major refactor to improve flexibility and extensibility while maintaining a user-friendly interface.
Some upcoming changes may be backward incompatible.
For details and migration guidance, visit: https://python.arviz.org/en/latest/user_guide/migration_guide.html
  warn(


In [5]:
PATH_MODEL = '/volumes/hyijie_psy/CPP_stage2_HDDM/high_Sun_2023/model_fitted'
PATH_MODEL_FIt = '../results/ddm'
os.makedirs(PATH_MODEL_FIt, exist_ok=True)

MODEL_LIST = {'m2_ams', 'm2_pams', 'm2_slps', 'm2_ams_bin', 'm2_pams_bin', 'm2_slps_bin'}
PARAMETER_NAME = {'v_ams', 'v_pams','v_slps','v_ams_bin','v_pams_bin','v_slps_bin'}

In [ ]:
# Extract and save posterior and Loo value

loo_results = []

for m , param_name in zip(MODEL_LIST, PARAMETER_NAME):
    path_model_infdata = os.path.join(PATH_MODEL, m + '.nc')
    assert os.path.exists(path_model_infdata), f" nc file not found: {path_model_infdata}"

    # Extract and save Loo for all models
    m_infdata = az.from_netcdf(path_model_infdata)
    loo = az.loo(m_infdata, pointwise=True)
    loo_dict = {
            "model": m,
            "loo": loo.loo,
        }
    loo_results.append(loo_dict)

    # Save posterior data for all models
    data_posterior = m_infdata.posterior[param_name].stack(sample=('chain', 'draw')).values
    data_posterior = pd.DataFrame({param_name: data_posterior})
    path_save_posterior = os.path.join(PATH_MODEL_FIt, m+'_posterior.csv')
    data_posterior.to_csv(path_save_posterior)

loo_df = pd.DataFrame(loo_results)
loo_df.to_csv(os.path.join(PATH_MODEL_FIt, "loo_comparison.csv"), index=False)